# <div style=" text-align: center; font-size: 50px; font-weight: bold">Phase 01: Collecting data </div>

## **Import necessary Python modules**

In [18]:
import pandas as pd 
from bs4 import BeautifulSoup
# !pip install selenium
from selenium import webdriver
# !pip install cloudscraper
import cloudscraper
import re

## **Collect real estate for sale data:**
In this section, we will collect the data of real estates that for sale from this website: https://batdongsan.com.vn/ . This webstite contain most of the advertisement of real estate (for rent or sale) in Viet Nam. In the scope of this project. We will focus on the data of real estate for sale, which are located in Ho Chi Minh City.

Also, with the heavy density of the advertisement in this website. The data is collected is just mainly concentrate in just a period time (just in round end months of 2023). So for a overview, they cannot give us a full view about the current state of real estate in Ho Chi Minh City over times. Anyway, just don't get more concern to it and start our journey to collect data.

### **1. Some initials:**

Look through the website, I can figure out some features that are common for most of real estate. So we can consider them like features that we should collect.

Now, create an empty dataframe to store the data of the real estate. The dataframe will contain some features like: `Address`, `Type`, `Area`, `Price`, `Bedroom`, `Toilet`, `Floor`, `Furniture`, `Direction`, `Legal`, `Posting date`, `Expiry date`, `Ad type`, `Ad code`.

In [19]:
fields = [ 'Address', 'Type', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

# Create a empty data frame with these fields
df = pd.DataFrame(columns = fields)

### **2. Collect the links to each real estate webpage:**

The links are collected from multiple pages, with each page containing numerous links to apartments. These links have been stored in a file named `real_estate_for_sale_links.txt`.     

With this task , we can use `Selenium` to access the website and crawl data from it. The link to the homepage and the file name are provided below:

In [20]:
file_name = "../Data/real_estate_for_sale_links.txt"
url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'

We will move from the first page to the last page to collect the links, so the first thing should do is get the final page of the website.

In [4]:
def GetTotalPage(url):

    # Create an instance of the driver
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # The parsing data process
    unprocessed_num_page = soup.find(class_ = 're__pagination-group')
    driver.quit()
    # Regex to find all the current numpages in the screen.
    regex = r'[^\s]+'
    matchs = re.findall(regex, unprocessed_num_page.text)
    new_array = []
    
    for match in matchs:
        new_array.append(int(match.replace('.', '')))

    total_page = max(new_array)

    return total_page

Process the result to get the last page.

In [23]:
# Get num of total pages in the website.
total_page = GetTotalPage(url)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00007FF7A295CF65+75717]
	GetHandleVerifier [0x00007FF7A295CFC0+75808]
	(No symbol) [0x00007FF7A2728F9A]
	(No symbol) [0x00007FF7A2701B41]
	(No symbol) [0x00007FF7A27AEC2E]
	(No symbol) [0x00007FF7A27CECC2]
	(No symbol) [0x00007FF7A27A7153]
	(No symbol) [0x00007FF7A2770421]
	(No symbol) [0x00007FF7A27711B3]
	GetHandleVerifier [0x00007FF7A2C5D74D+3223469]
	GetHandleVerifier [0x00007FF7A2C57CF2+3200338]
	GetHandleVerifier [0x00007FF7A2C75B23+3322755]
	GetHandleVerifier [0x00007FF7A2976A3A+180890]
	GetHandleVerifier [0x00007FF7A297E13F+211359]
	GetHandleVerifier [0x00007FF7A29652B4+109332]
	GetHandleVerifier [0x00007FF7A2965462+109762]
	GetHandleVerifier [0x00007FF7A294BA79+4825]
	BaseThreadInitThunk [0x00007FFA492C259D+29]
	RtlUserThreadStart [0x00007FFA4AFAAF38+40]


Function to collect the links to each real estate:

In [ ]:
def CollectRealEstateForSaleLink (url, page):
    with open (file_name, 'w') as file:
        # Acess to all page
        for i in range(1, page + 1):
            driver = webdriver.Chrome()
            driver.get(url+'/p'+ str(i))
            soup = soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

            # Add all links to the file
            for apartment in apartment_list:
                link = 'https://batdongsan.com.vn/'+ apartment['href']
                file.write(link)
                file.write('\n')

            driver.quit()

In [ ]:
CollectRealEstateForSaleLink (url, total_page)

Exception ignored in: <function Service.__del__ at 0x000001B0CA03C680>
Traceback (most recent call last):
  File "c:\Users\bkphu\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
    self.stop()
  File "c:\Users\bkphu\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 157, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\bkphu\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\bkphu\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bkphu\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bkphu\anaconda3\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open

Up to now, we have collected all the links to each real estate website and store them into a text file. 

Read the `real_estate_for_sale_links.txt` file and store all the links to a lists

In [21]:
urls = []
with open(file_name, 'r') as file:
    for line in file.readlines():
            urls.append(line)

In [22]:
len(urls)

65358

### **3. Access to each link and colllect the data:**

The current language of the website is Vietnamese, and as just for some consitent, I try to convert them all to Englist by the dictionary below:

In [23]:
key_map = {
    'Địa chỉ:' : 'Address',
    'Diện tích': 'Area',
    'Mức giá': 'Price',
    'Hướng nhà': 'Direction',
    'Số tầng': 'Floor',
    'Số phòng ngủ': 'Bedroom',
    'Số toilet': 'Toilet',
    'Pháp lý': 'Legal',
    'Nội thất': 'Furniture',
    'Ngày đăng': 'Posting date',
    'Ngày hết hạn': 'Expiry date',
    'Loại tin': 'Ad type',
    'Mã tin': 'Ad code'
}

Also, look the through the website, I see that there are so many different types of real estates. This will be a important features for out future analysis. The issue here is the current data of each real estate that not contain its type, and the type is hide by the type code. Fortunaly, I have found all of them and store in the dictionary bellow:

In [24]:
sale_type = ['Căn hộ chung cư', 'Nhà riêng', 'Nhà biệt thự, liền kề', 'Nhà mặt phố', 'Shophouse nhà phố thương mại', 'Đất nền dự án', 'Đất bán', 'Trang trại, khu nghỉ dưỡng', 'Kho, nhà xưởng','Bất động sản khác','Condotel','kháckhác']
sale_type_dict = {
    '324': sale_type[0], # Chung cư
    '41': sale_type[1], # Nhà riêng
    '325': sale_type[2], # biệt thự
    '163': sale_type[3], # Nhà mặt phố
    '575': sale_type[4], # Shophouse
    '40': sale_type[5], # Đất nền
    '283': sale_type[6], # Bán đất
    '44': sale_type[7], # Trang trại
    '45': sale_type[8], # kho
    '48': sale_type[9], # Khác
    '562': sale_type[10], # Khác
    '650': sale_type[10]
}


Now, let's define some necessary function of scrapping process:

- Function to extract the features from a link:

In [25]:
def ExtractDataFromLinkWithSelenium(url):
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Create a dicitonary to store the feature
        features = {}

        # get the address
        address = soup.find(class_ = "re__pr-short-description js__pr-address")

        # Get the type
        regex = r'[\d]+'
        type = soup.find_all('script', type= 'text/javascript')
        for item in type:
            if (item.text):
                found = re.findall(regex, item.text)
                break;
        # If the address cannot be found, mean the data is invalide, ignore them.
        if address == None:
            return features
        else:
            features['Address'] = address.text
            features['Type'] = sale_type_dict[found[3]]

        # Get all the features about the attribute of each real estate
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        # Get the features about the posting date, expiry date and the ad code
        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value
    
        # Convert all the keys to the defined keys base on the dictionary.
        keys_to_change = list(features.keys())
        for key in keys_to_change:
            if key in key_map:
                features[key_map[key]] = features[key]
    finally:
        driver.quit()

    return features

Iterate over the urls list and extract the data, which is stored in the previously defined DataFrame

In [26]:
for url in urls[27000:30000]:
        features = ExtractDataFromLinkWithSelenium(url)
        if features:
                new_row = pd.Series(features, index = fields)
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df], ignore_index = True)
                #new_row_df.to_csv('../Data/real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')
        else:
                print ("Error")


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [27]:
df

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Đường Tân Chánh Hiệp 8, Phường Tân Chánh Hiệp,...",Nhà mặt phố,145 m²,"13,88 tỷ",5 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,15/05/2025,25/05/2025,Tin thường,42783626
1,"Đường Phạm Thế Hiển, Phường 7, Quận 8, Hồ Chí ...",Nhà mặt phố,130 m²,"10,8 tỷ",NaN,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,22/05/2025,01/06/2025,Tin thường,42864948
2,"Đường Hiệp Bình, Phường Hiệp Bình Chánh, Thủ Đ...",Nhà mặt phố,118 m²,"16,9 tỷ",4 phòng,NaN,3 tầng,Tặng nội thất.,NaN,Sổ đỏ/ Sổ hồng,23/05/2025,07/06/2025,Tin thường,39890261
3,"Đường 27, Phường Hiệp Bình Chánh, Thủ Đức, Hồ ...",Đất bán,188 m²,"17,8 tỷ",NaN,NaN,NaN,NaN,Đông - Nam,Sổ đỏ/ Sổ hồng.,17/05/2025,01/06/2025,Tin thường,42576243
4,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,19/05/2025,29/05/2025,Tin thường,42374969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2929,"Dự án The Sun Residence, Đường Đào Tông Nguyên...",Nhà riêng,50 m²,"5,7 tỷ",4 phòng,NaN,4 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng.,22/05/2025,01/06/2025,Tin thường,35818525
2930,"Đường Huỳnh Tấn Phát, Thị trấn Nhà Bè, Nhà Bè,...",Nhà riêng,80 m²,"9,2 tỷ",4 phòng,NaN,3 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,19/05/2025,29/05/2025,Tin thường,41524321
2931,"Đường D1, Phường Phú Mỹ, Quận 7, Hồ Chí Minh",Nhà riêng,"92,5 m²","9,1 tỷ",4 phòng,NaN,3 tầng,Full.,NaN,Sổ đỏ/ Sổ hồng.,23/05/2025,02/06/2025,Tin thường,39351252
2932,"Đường Liên khu 10-11, Phường Bình Hưng Hòa A, ...",Nhà riêng,96 m²,Thỏa thuận,2 phòng,NaN,2 tầng,"- Nội thất đầy đủ: Điều hòa, tủ lạnh, giường...",NaN,Sổ đỏ/ Sổ hồng,01/05/2025,11/05/2025,Tin thường,42862524


### **4. Store the collected data to csv file:**

Finally, just store the data into a `csv` file and finish the process.

In [28]:
df.to_csv('../Data/real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')

In [29]:
real_estate_df = pd.read_csv('../Data/real_estate_for_sale.csv')
real_estate_df

,Address,Type,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Đường Lê Văn Việt, Phường Long Thạnh Mỹ, Quận ...",Nhà riêng,66 m²,Thỏa thuận,4 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,42376273
1,"Dự án Fuji Residence, Đường Đỗ Xuân Hợp, Phườn...",Căn hộ chung cư,55 m²,"2,7 tỷ",1 phòng,NaN,NaN,Đầy đủ,Tây - Bắc,Sổ đỏ/ Sổ hồng,03/05/2025,13/05/2025,Tin thường,41283793
2,"Đường Lê Văn Sỹ, Phường 1, Tân Bình, Hồ Chí Minh",Nhà riêng,45 m²,8 tỷ,4 phòng,NaN,4 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,08/05/2025,18/05/2025,Tin thường,42374969
3,"Đường Sư Vạn Hạnh, Phường 12, Quận 10, Hồ Chí ...",Nhà riêng,109 m²,"23,8 tỷ",5 phòng,NaN,3 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng,15/05/2025,25/05/2025,Tin thường,42342057
4,"Đường Bạch Đằng, Phường 24, Bình Thạnh, Hồ Chí...",Nhà mặt phố,52 m²,"8,9 tỷ",NaN,NaN,3 tầng,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,04/05/2025,14/05/2025,Tin thường,42571965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37460,"Dự án The Sun Residence, Đường Đào Tông Nguyên...",Nhà riêng,50 m²,"5,7 tỷ",4 phòng,NaN,4 tầng,NaN,NaN,Sổ đỏ/ Sổ hồng.,22/05/2025,01/06/2025,Tin thường,35818525
37461,"Đường Huỳnh Tấn Phát, Thị trấn Nhà Bè, Nhà Bè,...",Nhà riêng,80 m²,"9,2 tỷ",4 phòng,NaN,3 tầng,Đầy đủ,NaN,Sổ đỏ/ Sổ hồng,19/05/2025,29/05/2025,Tin thường,41524321
37462,"Đường D1, Phường Phú Mỹ, Quận 7, Hồ Chí Minh",Nhà riêng,"92,5 m²","9,1 tỷ",4 phòng,NaN,3 tầng,Full.,NaN,Sổ đỏ/ Sổ hồng.,23/05/2025,02/06/2025,Tin thường,39351252
37463,"Đường Liên khu 10-11, Phường Bình Hưng Hòa A, ...",Nhà riêng,96 m²,Thỏa thuận,2 phòng,NaN,2 tầng,"- Nội thất đầy đủ: Điều hòa, tủ lạnh, giường...",NaN,Sổ đỏ/ Sổ hồng,01/05/2025,11/05/2025,Tin thường,42862524


In [ ]:
real_estate_df.shape()